## Trajectory tutorial  - 2022 0127 CJH
* We need to generate a list of characteristics for the robot to track, bundled into a trajectory.  
* Trajectories give you:
  * A list of states with t, velocity, acceleration, pose=Pose2d(Translation2d(x, y), Rotation2d(rad)), curvature)
  * A sample() method to get the state at any time
  * A totalTime member
  * a transformBy so we can change the starting point
  
* We need input to the trajectory - the points it must follow
  * pathweaver **paths** (not to be confused with python *Paths* or operating systems directories) do this
  * we can do it with lists of points as well


---

#### Auto generating trajecotories to follow using wpilib


In [1]:
import glob
from pathlib import Path

from wpimath.geometry import Pose2d, Rotation2d, Translation2d, Transform2d, Twist2d
import wpimath.trajectory
from wpimath.kinematics import DifferentialDriveKinematics
from wpimath.trajectory.constraint import DifferentialDriveVoltageConstraint, CentripetalAccelerationConstraint
import wpilib.controller

ModuleNotFoundError: No module named 'wpilib.controller'

#### unfortunately, trajectories need to know a lot about the robot

In [ ]:
# All the characterization constants and drive attributes we need to generate a trajectory
ks_volts = 0.446
kv_volt_seconds_per_meter = 1.55
ka_volt_seconds_squared_per_meter = 0.40
k_track_width_meters = 24 * 0.0254
k_max_voltage = 6
k_max_centripetal_acceleration_meters_per_second_squared = 2.75
k_max_acceleration_meters_per_second_squared = 2.75
k_drive_kinematics = DifferentialDriveKinematics(k_track_width_meters)
k_feed_forward = wpilib.controller.SimpleMotorFeedforwardMeters(ks_volts, kv_volt_seconds_per_meter, ka_volt_seconds_squared_per_meter)
k_autonomous_voltage_constraint = DifferentialDriveVoltageConstraint(k_feed_forward, k_drive_kinematics, k_max_voltage)

#### the most basic trajectory - connect two poses

In [ ]:
start_pose = Pose2d(Translation2d(x=0, y=0), Rotation2d(0.000000))
end_pose = Pose2d(Translation2d(x=1, y=0), Rotation2d(0))
velocity = 1
config = wpimath.trajectory.TrajectoryConfig(velocity, k_max_acceleration_meters_per_second_squared)
traj = wpimath.trajectory.TrajectoryGenerator.generateTrajectory(waypoints=[start_pose, end_pose], config=config)
traj.states()

#### the next way of making trajectories

In [ ]:
len(traj.states())

In [ ]:
traj_2 = wpimath.trajectory.TrajectoryGenerator.generateTrajectory?

In [ ]:
traj_2 = wpimath.trajectory.TrajectoryGenerator.generateTrajectory

In [ ]:
traj_2 = wpimath.trajectory.TrajectoryGenerator.generateTrajectory

#### the full on, on-the fly version of generating trajectories on the robot

In [ ]:
pwd

In [ ]:
path_location = r'..\..\..\2022\robot\pathweaver\paths\\'
# find all of the pathweaver paths available to the robot - use to populate a drop-down sendable chooser in the UI
def get_pathweaver_paths():  # use this to fill the drop down for file selection
    # instead of passing robot simulation, check for directory exists on the robot
    path_files = glob.glob(path_location + '*', recursive=True)
    #print(f'** Pathweaver files: {path_files} **')
    path_names = [Path(file).name for file in path_files]
    return path_names

In [ ]:
paths = get_pathweaver_paths()
paths

In [ ]:
Path(path_location) / paths[0]

In [ ]:
def generate_trajectory(path_name:str, velocity=2, display=True, save=True) -> wpimath.trajectory:
    """
    Generate a wpilib trajectory from a pathweaver path.  Accepts regular and reversed paths.

    :param path_name: name of pathweaver file to be imported
    :param velocity: Maximum robot velocity for the generated trajectory
    :param save: Option to save generated trajectory to disk as 'test.json'
    :param display: Option to print to console
    :return: generated trajectory
    """

    pathweaver_y_offfset = 4.572
    p = Path(path_location + path_name)
    if p.is_file():
        lines = []
        with open(p, "r") as f:
            for line in f:
                currentline = line.split(",")
                lines.append(currentline)
        cvector_list = [wpimath.spline.Spline5.ControlVector((float(row[0]), float(row[2]), 0), (float(row[1]) + pathweaver_y_offfset, float(row[3]), 0))
                        for ix, row in enumerate(lines[1:])]

        config = wpimath.trajectory.TrajectoryConfig(velocity, k_max_acceleration_meters_per_second_squared)
        config.setKinematics(k_drive_kinematics)
        config.addConstraint(k_autonomous_voltage_constraint)
        config.addConstraint(CentripetalAccelerationConstraint(k_max_centripetal_acceleration_meters_per_second_squared))
        # check to see if the path is to be reversed it is marked in the th column of the pathweaver file
        reverse_array = [row[5] for row in lines[1:]]
        if any(entry == 'true' for entry in reverse_array):
            config.setReversed(True)
        pw_trajectory = wpimath.trajectory.TrajectoryGenerator.generateTrajectory(cvector_list, config)
        if save:
            wpimath.trajectory.TrajectoryUtil.toPathweaverJson(pw_trajectory, 'pathweaver\\test.json')
    else:
        pw_trajectory = None  # do something else? generate an empty trajectory?
        print(f'Trajectory: {p} not found', flush=True)
    if display:
        print(pw_trajectory)
    return pw_trajectory

In [ ]:
traj = generate_trajectory(path_name=paths[1], velocity=2, display=True, save=False)

In [ ]:
traj_attribs = [attrib for attrib in dir(traj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(traj_attribs) ]

In [ ]:
traj.initialPose()

In [ ]:
traj.sample(t=1.5)

In [ ]:
# show all of the states of the trajectory
traj.states()

In [ ]:
traj.states()[-1].pose

#### let's move the trajectory

In [ ]:
traj.transformBy?

In [ ]:
new_start_pose = Pose2d(Translation2d(0, 0), Rotation2d(0)) 

In [ ]:
# can just do it directly
transform = Transform2d(Translation2d(x=-1, y=-0.572), Rotation2d(0))
transform

In [ ]:
# or can use the initial and final poses
Transform2d(traj.initialPose(), new_start_pose)

In [ ]:
new_traj = traj.transformBy(transform)

In [ ]:
new_traj.states()  # now we start at zero